# OpenAI - Real Time API with tool calling

## Overview

The audio file *./assets/audio/QuestionSuperSportChampionship2025.wav* contains the question ***Who has won the Super Sports Championship 2025?*** and will be provided directly to the model instance. Because it's a fictional sport event the model will not be able to answer the question without further grounding information or function/tool calling.

## Environment

An Azure OpenAI GPT-4 instance (gpt-4o-mini-realtime-preview) will be used in this sample. The necessary connection information is imported from the ***config.env*** file in the [../config/ folder](../config). The provided [Azure CLI Powershell script](../setup/setup.azcli) can be used to create the necessary instance.



In [1]:
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.2"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: NAudio, 2.2.1"

using DotNetEnv;
using System.IO;
using Azure.AI.OpenAI;
using OpenAI.RealtimeConversation;
using System.ClientModel;
using NAudio.Wave;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("RT_OPENAI_APIKEY");
string endpoint = Env.GetString("RT_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("RT_OPENAI_CHATCOMPLETION_DEPLOYMENT");
string assetFolder = "../assets/";

string audioFile = Path.Join(assetFolder, "Audio", "QuestionSuperSportChampionship2025.wav");

Console.WriteLine($"Configuration loaded...");

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.2 DotNetEnv, 3.1.1 NAudio, 2.2.1

Configuration loaded...


## Tool Definition

The `RetrieveSportResultsTool` method defines a `ConversationFunctionTool` named "*get_sport_event_result*", which is designed to retrieve the result of a specified sports event. 

It includes a description explaining its purpose and specifies a JSON schema for its parameters which can be called by the model to achieve the expected outcome.


In [3]:
#pragma warning disable OPENAI002 

static ConversationFunctionTool RetrieveSportResultsTool()
{
    return new ConversationFunctionTool("get_sport_event_result")
    {
        Description = "gets the result of a sport event",
        Parameters = BinaryData.FromString("""
        {
            "type": "object",
            "properties": {
            "event": {
                "type": "string",
                "description": "The sport event, e.g. Super Bowl"
            }
            },
            "required": ["event"]
        }
        """)
    };
}

Console.WriteLine($"ConversationFunctionTool defined...")

ConversationFunctionTool defined...


## Realtime Conversation Client

Azure OpenAI SDK provides an `RealTimeConversationClient` to simplify the interaction with the model's real time API.

In [4]:
#pragma warning disable OPENAI002

ApiKeyCredential azureKeyCredential = new ApiKeyCredential(apiKey);
AzureOpenAIClient openAIClient = new AzureOpenAIClient(new Uri(endpoint), azureKeyCredential);
RealtimeConversationClient realtimeConversationClient = openAIClient.GetRealtimeConversationClient(chatCompletionDeployment);

Console.WriteLine($"Realtime conversation client created...");

Realtime conversation client created...


## Real time conversation session

An instance of `ConversationSessionOptions` is used to configure a specific session with the model. It allows providing tools to support the model in it's task by providing existing functions to the `Tools` property.

```csharp
    ConversationSessionOptions conversationSessionOptions = new ConversationSessionOptions(){
        ...,
        Tools = {RetrieveSportResultsTool()},
        ...
    };
```

In [5]:
#pragma warning disable OPENAI002

string systemPrompt = "You are an assistant which helps user to find information about sport events.";

RealtimeConversationSession realTimeConversationSession = await realtimeConversationClient.StartConversationSessionAsync();
ConversationSessionOptions conversationSessionOptions = new ConversationSessionOptions(){
    Instructions = systemPrompt,
    Voice = ConversationVoice.Alloy,
    Tools = {RetrieveSportResultsTool()},
    InputAudioFormat = ConversationAudioFormat.Pcm16,
    OutputAudioFormat = ConversationAudioFormat.Pcm16,
    InputTranscriptionOptions = new ConversationInputTranscriptionOptions(){
        Model = "whisper-1",
    }
};
await realTimeConversationSession.ConfigureSessionAsync(conversationSessionOptions); 

Console.WriteLine($"Realtime conversation session configured...");


Realtime conversation session configured...


## Conversation input data

Within a conversation multi modal input data can be provided. Within this sample text and audio data is provided.

In [6]:
#pragma warning disable OPENAI002

await realTimeConversationSession.AddItemAsync(
    ConversationItem.CreateUserMessage(new ConversationContentPart[] {"Please answer in German!"})
);

await realTimeConversationSession.SendInputAudioAsync(new FileStream(audioFile, FileMode.Open, FileAccess.Read));

Console.WriteLine($"Conversation session updated with text + audio data...");

Conversation session updated with text + audio data...


## Model Response

The SDK function `RealTimeConversationSession.ReceiveUpdatesAsync()` provides  asynchronous updates of the processing. 

- `ConversationUpdateKind.ItemStreamingPartAudioDelta`: Audio response from the model. In the simplified sample the audio response is stored in *.wav files.
- `ItemStreamingFinishedUpdate.FunctionCallId`: indicates that a tool (function) call is requested by the model.



In [ ]:
#pragma warning disable OPENAI002

int updateCount = 1;

await foreach (ConversationUpdate conversationUpdate in realTimeConversationSession.ReceiveUpdatesAsync())
{
    if (conversationUpdate is ConversationItemStreamingPartDeltaUpdate conversationItemStreamingPartDeltaUpdate)
    {
        if (conversationUpdate.Kind == ConversationUpdateKind.ItemStreamingPartAudioDelta) 
        {                        
            Stream? audioUpdateFromModel = conversationItemStreamingPartDeltaUpdate?.AudioBytes.ToStream(); 
            if (audioUpdateFromModel is not null)
            {
                // Handle audio response from model (e.g. stream to speaker, save to file, etc.)
                // In this example, raw audio response is converted to wav file
                string fileName = Path.ChangeExtension(audioFile, $".response{updateCount++}.wav");
                
                WaveFormat waveFormat = new WaveFormat(24000, 16, 1);
                using RawSourceWaveStream rawSourceWaveStream = new RawSourceWaveStream(audioUpdateFromModel, waveFormat);
                using WaveFileWriter waveFileWriter = new WaveFileWriter(fileName, waveFormat);
                rawSourceWaveStream.CopyTo(waveFileWriter);
                
                Console.WriteLine($"\tAudio update from model written to file: {fileName}");
            }
        }
    }

    if (conversationUpdate is ConversationItemStreamingFinishedUpdate itemStreamingFinishedUpdate)
    {
        if (itemStreamingFinishedUpdate.FunctionCallId is not null)
        {
            //Tool call request identified
            //Simulate tool call and provide response
            Console.WriteLine($"Tool Call requested: {itemStreamingFinishedUpdate.FunctionName}");
            
            //Simulated tool call result
            string simulatedToolCallResult = "Munich Flying Dolphins with a score of 21 to 14";

            ConversationItem functionOutputItem = ConversationItem.CreateFunctionCallOutput(
                callId: itemStreamingFinishedUpdate.FunctionCallId,
                output: simulatedToolCallResult
            );

            await realTimeConversationSession.AddItemAsync(functionOutputItem);
        }
        else if (itemStreamingFinishedUpdate.MessageContentParts?.Count > 0)
        {
            foreach (ConversationContentPart contentPart in itemStreamingFinishedUpdate.MessageContentParts)
            {
                Console.WriteLine($"Audio transcript: {contentPart.AudioTranscript}");
            }
        }
    }

    if (conversationUpdate is ConversationResponseFinishedUpdate turnFinishedUpdate)
    {
        Console.WriteLine($"Model turn generation finished");
        if (turnFinishedUpdate.CreatedItems.Any(item => item.FunctionName?.Length > 0))
        {
            await realTimeConversationSession.StartResponseAsync();
        }
        else
        {
            break;
        }
    }
}

Tool Call requested: get_sport_event_result
Model turn generation finished
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response1.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response2.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response3.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response4.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response5.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response6.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response7.wav
	Audio update from model written to file: ../assets/Audio\QuestionSuperSportChampionship2025.response8.wav
	Audio update from model written to file: ../assets/Audio\QuestionSup

## Housekeeping

In [9]:
realTimeConversationSession.Dispose();

Console.WriteLine($"Realtime conversation session disposed...");


Realtime conversation session disposed...
